In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
# !unzip dataset.zip

In [ ]:
train_path="dataset/train"
test_path="dataset/test"

In [ ]:
# os.remove("dataset/not_logging/.ipynb_checkpoints")
# os.remove("/content/dataset/train/not_logging/.ipynb_checkpoints")
import shutil
for root, subdirs, files in os.walk('.'):
    for d in subdirs:
        if d == ".ipynb_checkpoints":
            shutil.rmtree(os.path.join(root, d))

In [ ]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):
        

        image_path=sub_path+"/"+img
        # print("img :", image_path)

        img_arr=cv2.imread(image_path)
      
        img_arr=cv2.resize(img_arr,(224,224))
        
        x_train.append(img_arr)

In [ ]:
x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):
        print("img : ", img)

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

img :  water logging in school_4.jpeg
img :  water logging in school_3.jpeg
img :  water logging in school_7.png
img :  water logging in school_5.jpeg
img :  water logging in school_6.jpeg
img :  indian village school campus_5.jpeg
img :  indian village school campus_3.jpeg
img :  indian village school campus_6.jpeg
img :  indian village school campus_4.jpeg
img :  indian village school campus_7.jpeg


In [ ]:
train_x=np.array(x_train)
test_x=np.array(x_test)

In [ ]:
train_x=train_x/255.0
test_x=test_x/255.0

In [ ]:
train_x.shape

(89, 224, 224, 3)

In [ ]:
test_x.shape

(10, 224, 224, 3)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 89 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
train_y=training_set.classes
test_y=test_set.classes

In [ ]:
training_set.class_indices
train_y.shape,test_y.shape

((89,), (10,))

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [ ]:
history = model.fit(
  train_x,
  train_y,
  validation_data=(test_x,test_y),
  epochs=30,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Epoch 1/30
3/3 [==============================] - 78s 26s/step - loss: 1.0790 - accuracy: 0.4270 - val_loss: 0.6629 - val_accuracy: 0.7000
Epoch 2/30
3/3 [==============================] - 77s 26s/step - loss: 0.3897 - accuracy: 0.8202 - val_loss: 0.8606 - val_accuracy: 0.6000
Epoch 3/30
3/3 [==============================] - 77s 27s/step - loss: 0.3204 - accuracy: 0.8539 - val_loss: 0.2959 - val_accuracy: 0.9000
Epoch 4/30
3/3 [==============================] - 74s 25s/step - loss: 0.1207 - accuracy: 0.9326 - val_loss: 0.2422 - val_accuracy: 0.9000
Epoch 5/30
3/3 [==============================] - 77s 26s/step - loss: 0.0894 - accuracy: 0.9438 - val_loss: 0.2630 - val_accuracy: 0.9000
Epoch 6/30
3/3 [==============================] - 77s 27s/step - loss: 0.0251 - accuracy: 1.0000 - val_loss: 0.4419 - val_accuracy: 0.8000
Epoch 7/30
3/3 [==============================] - 76s 27s/step - loss: 0.0357 - accuracy: 1.0000 - val_loss: 0.5122 - val_accuracy: 0.8000
Epoch 8/30
3/3 [===========